In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import akshare as ak
import datetime
import glob
import numpy as np

In [19]:
def get_data(index="sh510050"):
    sh50_files = sorted(glob.glob("../sh_index_data_tdq/{}_*.tsv".format(index)))

    sh50_pd = pd.read_table(sh50_files[0], header=0, index_col=0,parse_dates=["day"])
    for f in sh50_files[1:]:
        add_pd = pd.read_table(f, header=0, index_col=0,parse_dates=["day"])
        add_pd = add_pd[add_pd["day"] > list(sh50_pd["day"])[-1]] 
        sh50_pd = sh50_pd.append(add_pd)
        
    if index == "sh512100":
        sh50_pd.loc[sh50_pd["datetime"].map(str) <"2022-09-05", "close"] *= 2.6895

    sh50_pd.reset_index(drop=True, inplace=True)
    sh50_pd["close_yes"] = sh50_pd["close"].shift()
    sh50_pd = sh50_pd[sh50_pd["datetime"].map(str) > "2021-03-18"]
    return sh50_pd

In [6]:
def finish_data(data_df):
    data_df["close_2hour"] = data_df["close"].shift(-24)
    data_df["close_1day"] = data_df["close"].shift(-48)
    data_df["close_2day"] = data_df["close"].shift(-48*2)
    data_df["close_3day"] = data_df["close"].shift(-48*3)
    data_df["close_5day"] = data_df["close"].shift(-48*5)
    data_df["close_10day"] = data_df["close"].shift(-48*10)

    
    return data_df

In [7]:
def day_data(data_df):
    first5min_pd = data_df.iloc[0::48, :]
    first5min_pd = first5min_pd.reset_index()
    first5min_pd["volume_change"] = first5min_pd["vol"].pct_change()
    first5min_pd["close_change_yes"] = (first5min_pd["open"] - first5min_pd["close_yes"])/first5min_pd["close_yes"]
    first5min_pd["close_change_2h"] = (first5min_pd["close_2hour"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change1"] = (first5min_pd["close_1day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change2"] = (first5min_pd["close_2day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change3"] = (first5min_pd["close_3day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change5"] = (first5min_pd["close_5day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change10"] = (first5min_pd["close_10day"] - first5min_pd["close"])/first5min_pd["close"]
    
    return first5min_pd

In [8]:
def mean_value(first5min_pd):
    #每次交易收益率
    a02 = np.mean(first5min_pd["close_change_2h"])
    a1 = np.mean(first5min_pd["close_change1"])
    a2 = np.mean(first5min_pd["close_change2"])
    a3 = np.mean(first5min_pd["close_change3"])
    a5 = np.mean(first5min_pd["close_change5"])
    a10 = np.mean(first5min_pd["close_change10"])
    return [a02, a1, a2, a3, a5, a10]

In [9]:
def get_extre(index="sz159915"):
    sh50_pd = get_data(index=index)
    sh50_pd = finish_data(sh50_pd)
    day_data1 = day_data(sh50_pd)
    a = len(day_data1)
    origin = mean_value(day_data1)
    # 策略获取首5分钟内放量超过10%的交易
    first5min_choosefangliang_pd = day_data1[day_data1["volume_change"] > 0.05 ]
#     first5min_choosefangliang_pd = first5min_choosefangliang_pd[first5min_choosefangliang_pd["close"] > first5min_choosefangliang_pd["open"] ]
    #高开  1%
    first5min_choosefangliang_pd = first5min_choosefangliang_pd[first5min_choosefangliang_pd["close_change_yes"] < -0.005 ]
 
    choose = mean_value(first5min_choosefangliang_pd)
    b = len(first5min_choosefangliang_pd)
    d = ";".join(list(first5min_choosefangliang_pd["day"].map(str)))
    return([x-y for x,y in zip(choose, origin)] + [a, b, d])

In [66]:
def get_pct_oneday(index="sz159915"):
    sh50_pd = get_data(index=index)
    sh50_pd = finish_data(sh50_pd)
    day_data1 = day_data(sh50_pd)
    day_data1["mean_5"] = day_data1["close"].rolling(5, min_periods=1).mean()
    day_data1["mean_20"] = day_data1["close"].rolling(20, min_periods=1).mean()
    day_data1["is_up"] = day_data1["mean_5"] > day_data1["mean_20"]
    
    get = "get_" + index
    day_data1[get] = 0
    
    # 上升下降不同策略
#     choose_up = (day_data1["volume_change"] > 0.01) & (day_data1["close_change_yes"] < -0.005) & day_data1["is_up"]
#     choose_down = (day_data1["volume_change"] > 0.05) & (day_data1["close_change_yes"] < -0.005) & (day_data1["is_up"] == False)
#     choose_up = choose_up | choose_down
#     choose_all = choose_up & day_data1["is_up"]

    choose_all = (day_data1["volume_change"] > 0.05) & (day_data1["close_change_yes"] < -0.005)
    choose_nextday = choose_all.shift().fillna(False)
    day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]

    return day_data1

In [41]:
tmp = get_pct_oneday(index="000016")

/tmp/ipykernel_3810998/1055765510.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]


In [43]:
mean_value

,index,open,close,high,low,vol,amount,year,month,day,...,close_change_2h,close_change1,close_change2,close_change3,close_change5,close_change10,mean_5,mean_20,is_up,get_000016
0,240,3598.85,3599.25,3610.73,3597.34,46280760.0,4.628079e+09,2021,3,18,...,-0.000353,-0.009821,-0.019910,-0.020274,-0.041328,-0.012061,3599.250,3599.2500,False,0.000000
1,288,3565.09,3563.90,3566.59,3552.12,62298344.0,6.229838e+09,2021,3,19,...,-0.009327,-0.010188,-0.010556,-0.016521,-0.018657,0.009549,3581.575,3581.5750,False,-0.010188
2,336,3516.66,3527.59,3527.59,3511.72,44109264.0,4.410929e+09,2021,3,22,...,0.000561,-0.000371,-0.006398,-0.021853,0.001777,0.023968,3563.580,3563.5800,False,0.000000
3,384,3541.33,3526.28,3542.15,3525.52,36808704.0,3.680873e+09,2021,3,23,...,-0.011806,-0.006029,-0.021490,-0.008187,0.005286,0.013961,3554.255,3554.2550,False,0.000000
4,432,3507.55,3505.02,3513.07,3502.46,35430280.0,3.543031e+09,2021,3,24,...,-0.009854,-0.015555,-0.002171,0.008228,0.014274,0.011469,3544.408,3544.4080,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,23328,2637.82,2651.58,2653.10,2637.71,48546560.0,4.854661e+09,2023,3,13,...,0.004265,0.000453,0.004284,-0.009689,NaN,NaN,2688.460,2740.5525,False,0.000000
482,23376,2658.28,2652.78,2658.28,2650.95,37724208.0,3.772425e+09,2023,3,14,...,-0.003868,0.003830,-0.010137,-0.003969,NaN,NaN,2670.000,2734.8440,False,0.000000
483,23424,2666.94,2662.94,2668.00,2660.32,43652560.0,4.365262e+09,2023,3,15,...,-0.004807,-0.013913,-0.007770,NaN,NaN,NaN,2662.824,2729.2990,False,0.000000
484,23472,2627.04,2625.89,2632.43,2622.34,48263088.0,4.826313e+09,2023,3,16,...,0.004220,0.006230,NaN,NaN,NaN,NaN,2650.856,2723.0955,False,0.006230


In [56]:
def get_extre(index="sz159915"):
    sh50_pd = get_data(index=index)
    sh50_pd = finish_data(sh50_pd)
    day_data1 = day_data(sh50_pd)
    a = len(day_data1)
    origin = mean_value(day_data1)
    # 策略获取首5分钟内放量超过10%的交易
    first5min_choosefangliang_pd = day_data1[day_data1["volume_change"] > 0.05 ]
#     first5min_choosefangliang_pd = first5min_choosefangliang_pd[first5min_choosefangliang_pd["close"] > first5min_choosefangliang_pd["open"] ]
    #高开  1%
    first5min_choosefangliang_pd = first5min_choosefangliang_pd[first5min_choosefangliang_pd["close_change_yes"] < -0.005 ]
 
    choose = mean_value(first5min_choosefangliang_pd)
    b = len(first5min_choosefangliang_pd)
    d = ";".join(list(first5min_choosefangliang_pd["day"].map(str)))
    return([x-y for x,y in zip(choose, origin)] + [a, b, d])

In [29]:
get_extre(index="000016")

[0.00032368383756354636,
 0.003164811468035724,
 0.00023785727658318036,
 0.0026810175722200734,
 0.004383418374692478,
 0.0013626780384128068,
 272,
 25,
 '28;26;28;5;22;25;29;22;4;7;24;28;6;22;25;22;26;24;31;7;21;28;6;27;8']

In [44]:
index_pd = get_data(index="000001")
index_pd = finish_data(index_pd)
day_data1 = day_data(index_pd)
day_data1 = day_data1[day_data1["datetime"] > "2021-03-18"]
day_data1

,index,open,close,high,low,vol,amount,year,month,day,...,close_5day,close_10day,volume_change,close_change_yes,close_change_2h,close_change1,close_change2,close_change3,close_change5,close_change10
0,480,3449.64,3451.29,3458.18,3449.38,247623952.0,2.476240e+10,2021,3,18,...,3356.65,3447.53,NaN,0.001187,0.003031,-0.006731,-0.009976,-0.004477,-0.027422,-0.001089
1,528,3423.87,3428.06,3429.16,3415.99,304866528.0,3.048665e+10,2021,3,19,...,3387.73,3476.17,0.231167,-0.011319,-0.002672,-0.003267,0.002270,-0.010082,-0.011765,0.014034
2,576,3406.12,3416.86,3416.86,3404.23,289159360.0,2.891594e+10,2021,3,22,...,3422.60,3485.26,-0.051521,0.000429,0.004984,0.005555,-0.006837,-0.017621,0.001680,0.020018
3,624,3445.34,3435.84,3445.42,3435.54,278157920.0,2.781579e+10,2021,3,23,...,3435.04,3473.73,-0.038046,0.000552,-0.011988,-0.012323,-0.023048,-0.014002,-0.000233,0.011028
4,672,3394.13,3393.50,3398.43,3391.69,258852080.0,2.588521e+10,2021,3,24,...,3443.43,3468.28,-0.069406,-0.005095,-0.007921,-0.010859,-0.001700,0.008575,0.014713,0.022036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,23568,3228.12,3241.85,3242.99,3228.12,324416928.0,3.244170e+10,2023,3,13,...,NaN,NaN,0.090423,-0.000607,0.003773,0.004976,0.005704,-0.000444,NaN,NaN
482,23616,3261.92,3257.98,3261.92,3255.99,299964960.0,2.999650e+10,2023,3,14,...,NaN,NaN,-0.075372,-0.002074,-0.006231,0.000724,-0.005393,-0.003877,NaN,NaN
483,23664,3261.03,3260.34,3264.12,3256.86,335141440.0,3.351415e+10,2023,3,15,...,NaN,NaN,0.117269,0.004844,0.000715,-0.006113,-0.004598,NaN,NaN,NaN
484,23712,3244.47,3240.41,3246.52,3237.84,342461888.0,3.424619e+10,2023,3,16,...,NaN,NaN,0.021843,-0.005773,0.001975,0.001524,NaN,NaN,NaN,NaN


In [67]:
# index_list = [
#     "000012","000016","000032","000033",
#     "000034","000035","000036","000037","000038",
#     "000039","000040","000041","000042","000043",
#     "000044","000045","000046","000047","000048",
#     "000049","000151","000152","000153","000155",
#     "000158","000159","000160","000161","000162",
#     "000300","000802","000819","000827","000852",
#     "000905"
# ]

index_list = [
    "sh510050", "sh512100", "sh563000", "sh510300", "sh510330", 
    "sh510500", "sh561550",
    "sz159845", "sz159949",  "sz159915", "sz159967", "sh588000", "sh588080",
    "sh588050", "sh515790", "sh512480", "sh512760", "sz159995", "sh512660", "sh516160", 
    "sh515030", "sh515700", "sh515050", "sh512880", "sh512000", 
    "sh512070", "sh512800", "sh512200", "sh510880", 
    "sz159825", "sz159867",  "sh515790", "sh512480", "sh512760", "sz159995", "sh512660", 
    "sh515030", "sh515700", "sz159611", "sh515050", "sh512880",
    "sh512000", "sh512070", "sh512800", "sh512200", "sh510880", "sz159905"
]
result = []
merge_df = day_data1[["datetime", "close"]]
for index in index_list:
    print(index)
    print(len(merge_df))
    a = get_pct_oneday(index=index)
    if len(a) < 90:
        continue
    index_df = a[["datetime", "get_" + index]]
    merge_df = pd.merge(left=merge_df, right=index_df, left_on=["datetime"], right_on=["datetime"])

000012
486
000016
486
000032
486
000033
486


/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choo

000034
486
000035
486
000036
486
000037
486


/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choo

000038
486
000039
486
000040
486
000041
486


...

/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choo

000300
486
000802
486
000819
486
000827
486


/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choo

000852
486
000905
486


/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_3810998/3090516847.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose_all | choose_nextday] = day_data1["close_change1"]


In [46]:
merge_df

,datetime,close,get_000012,get_000016,get_000032,get_000033,get_000034,get_000035,get_000036,get_000037,...,get_000159,get_000160,get_000161,get_000162,get_000300,get_000802,get_000819,get_000827,get_000852,get_000905
0,2021-03-18 09:35,3451.29,0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2021-03-19 09:35,3428.06,0,-0.010188,-0.004130,-0.004482,0.001550,0.0,-0.007877,-0.001548,...,-0.004387,0.002950,0.007487,0.004214,-0.009699,0.004584,0.000000,0.007273,0.004211,0.002875
2,2021-03-22 09:35,3416.86,0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2021-03-23 09:35,3435.84,0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.013459,0.000000,0.000000
4,2021-03-24 09:35,3393.50,0,0.000000,-0.001490,-0.026977,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,2023-03-13 09:35,3241.85,0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.003193,0.000000,0.000000,0.000000,0.000000
482,2023-03-14 09:35,3257.98,0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
483,2023-03-15 09:35,3260.34,0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
484,2023-03-16 09:35,3240.41,0,0.006230,-0.004645,-0.007030,-0.009479,0.0,0.000000,0.000000,...,0.000000,-0.000763,0.000000,0.000000,0.000000,-0.002261,-0.010473,0.000000,0.000000,-0.001186


In [68]:
mean_get = list()
len_get = list()
for day_raw in merge_df.iterrows():
    day_dict = day_raw[1]
    etfs = list()
    gets = list()
    for k,v in day_dict.items():
        if k.startswith("get") and  v != 0.0:
            etfs.append(k)
            gets.append(v)
    
    # print(etfs, gets)
    if len(gets) <=5:
        mean_get.append(0)
    else:
        mean_get.append(np.mean(gets))
    len_get.append(len(gets))

In [25]:
mean_get

[0,
 0.0013659633918327004,
 0.0044275782603090405,
 0,
 -0.016457641448735842,
 0.021056293546947144,
 0.01564230332937724,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.004178720064591684,
 -0.002013489471917306,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -0.005043077703635891,
 0.0188250325272285,
 0,
 0,
 0,
 0.00037578801648765185,
 -0.0014887165688288627,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.008871474544857512,
 0.0014100521224397965,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0016631387077703425,
 0.004608911481078967,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -0.011238171406257212,
 -0.0007442779117896621,
 0,
 0.002909313264100051,
 -0.01851239883572552,
 0,
 0,
 0,
 0,
 0,
 0,
 -0.003762199407457536,
 0,
 0,
 0,
 0,
 -0.025754724473442264,
 -0.028817521135363507,
 0.015535447267472939,
 0.004762470486992243,
 0,
 0.023179127651805846,
 0.005923050080739689,
 0,
 0.0059835669107472945,
 -0.002707292898091648,
 0.00751888539645759,
 0,
 0,
 0,
 0,
 -0.006595839689391257,
 0

In [69]:
merge_df["mean_get"] = mean_get
merge_df["len_get"] = len_get
merge_df["mean_get_cum"] = (merge_df["mean_get"] + 1).cumprod()
merge_df.to_csv("all_etf_get2.tsv", sep="\t")

In [33]:
ah01 = index_pd = get_data(index="000001")

In [36]:
ah01["mean_5"] = ah01["close"].rolling(5, min_periods=1).mean()
ah01["mean_20"] = ah01["close"].rolling(20, min_periods=1).mean()

In [38]:
ah01["is_up"] = ah01["mean_5"] > ah01["mean_20"]

In [49]:
ah01["is_up"] == False

480      True
481      True
482      True
483      True
484      True
         ... 
23803    True
23804    True
23805    True
23806    True
23807    True
Name: is_up, Length: 23328, dtype: bool